# Evolutionnary Hierarchical Dirichlet Processes for Multiple Correlated Time Varying Corpora

## Introduction 

-----------------

Le notebook suivant est l'implémentation du code de l'article EvoHDP, réalisé par J.Zhang,Y.Song & al et est testé sur les données synthétiques indiqués par l'article, accessible grâce au lien suivant : 
<br\>
http://www.shixialiu.com/publications/evohdp/paper.pdf
<br\> <br\> 
Les détails et rappels mathématiques sont donnés au fur et à mesure de la rédaction du code

Les références (telles que "voir Table x" ou "voir (xx)") sont celles utilisées dans l'article

-----------------

In [2]:
#import os 
#from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from scipy.stats import multinomial
from scipy.special import gammaln
import copy
import math
import mpmath

In [78]:
# les données sont organisées sous cette forme : data=[T][J][[doc_t_j_1],[doc_t_j_2],...]

# Experiments on synthetic data 

Les données synthétiques sont une mixture de multinomiales, de paramètres $\phi_k$ indiqué en Table 1 et repris ci-dessous.  

In [80]:
true_phi=np.zeros((8,2))
true_phi[0]=[0.1,0.9]
true_phi[1]=[0.2,0.8]
true_phi[2]=[0.3,0.7]
true_phi[3]=[0.4,0.6]
true_phi[4]=[0.5,0.5]
true_phi[5]=[0.6,0.4]
true_phi[6]=[0.7,0.3]
true_phi[7]=[0.8,0.2]
T=4
J=3
W=2
K=40

In [82]:
# On crée un liste Info_data_sample=[T][J][local_components,size_corpora]
corpora_sizes=[[500,300,400],[510,320,430],[520,320,430],[530,340,450]]
def local_components_and_corpora_sizes(T,J,corpora_sizes):
    info_data=[]
    for t in range(T):
        info_data_t=[]
        for j in range(J):
            info_data_j=[]
            for k in range(3):
                info_data_j.append(j+k+t)
            info_data_j.append(corpora_sizes[t][j])
            info_data_t.append(info_data_j)
        info_data.append(info_data_t)   
    return(info_data)
info_data=local_components_and_corpora_sizes(T,J,corpora_sizes)
info_data

[[[0, 1, 2, 500], [1, 2, 3, 300], [2, 3, 4, 400]],
 [[1, 2, 3, 510], [2, 3, 4, 320], [3, 4, 5, 430]],
 [[2, 3, 4, 520], [3, 4, 5, 320], [4, 5, 6, 430]],
 [[3, 4, 5, 530], [4, 5, 6, 340], [5, 6, 7, 450]]]

In [85]:
def mixture_of_three_multinomial(liste_of_phi_indices,true_phi,corpora_size,z):
    
    mult1=np.random.multinomial(200,true_phi[liste_of_phi_indices[0]],size=z[0]).tolist()
    mult2=np.random.multinomial(200,true_phi[liste_of_phi_indices[1]],size=z[1]).tolist()
    mult3=np.random.multinomial(200,true_phi[liste_of_phi_indices[2]],size=z[2]).tolist()
    mixt_mult_float=np.concatenate((np.concatenate((mult1,mult2),axis=0),mult3),axis=0)
    #print(mixt_mult_float)
    mixt_mult_int=[[mixt_mult_float[t][j].tolist() for j in range(len(mixt_mult_float[t]))] for t in range(len(mixt_mult_float))]
    return(mixt_mult_int)

def generate_data_from_mixture_of_multinomials(T,J,info_data,true_phi):
    data=[]
    for t in range(T):
        data_t=[]
        for j in range(J):
            z=np.random.multinomial(info_data[t][j][3],[1/3,1/3,1/3])           
            doc_t_j=mixture_of_three_multinomial(info_data[t][j],true_phi,info_data[t][j][3],z)
            data_t.append(doc_t_j)
        #data_t.append(data_j)
        data.append(data_t)
    return(data)
data=generate_data_from_mixture_of_multinomials(T,J,info_data,true_phi)
#data=[T][J][[doc_t_j_1],[doc_t_j_2],...]
data

[[[[27, 173],
   [17, 183],
   [20, 180],
   [18, 182],
   [22, 178],
   [13, 187],
   [25, 175],
   [21, 179],
   [15, 185],
   [10, 190],
   [35, 165],
   [19, 181],
   [17, 183],
   [17, 183],
   [17, 183],
   [28, 172],
   [18, 182],
   [27, 173],
   [25, 175],
   [20, 180],
   [18, 182],
   [24, 176],
   [17, 183],
   [14, 186],
   [17, 183],
   [33, 167],
   [19, 181],
   [16, 184],
   [26, 174],
   [18, 182],
   [16, 184],
   [19, 181],
   [26, 174],
   [17, 183],
   [21, 179],
   [17, 183],
   [17, 183],
   [18, 182],
   [19, 181],
   [31, 169],
   [19, 181],
   [30, 170],
   [18, 182],
   [19, 181],
   [24, 176],
   [16, 184],
   [17, 183],
   [21, 179],
   [22, 178],
   [23, 177],
   [21, 179],
   [27, 173],
   [23, 177],
   [24, 176],
   [23, 177],
   [27, 173],
   [23, 177],
   [21, 179],
   [20, 180],
   [25, 175],
   [17, 183],
   [24, 176],
   [19, 181],
   [19, 181],
   [20, 180],
   [20, 180],
   [27, 173],
   [23, 177],
   [19, 181],
   [19, 181],
   [17, 183],
   [16

L'objectif de cette expérimentation est de retrouver les "true_phi" par l'algorithme EvoHDP. Ces "true_phi" ont été utilisé pour générer nos données. 

# Initialize hyper parameters

Pour l'initialisation le modèle est celui d'un HDP à trois niveaux :

$$ H \sim Dir ( 1/W) $$ 
$$ G \sim DP(\xi , H) $$

Pour chaque temps :

$$ \forall t \in T $$
$$ G_{0}^t \sim DP(\gamma^t , G) $$

Pour chaque corpus : 

$$ \forall j \in J $$
$$ G_{j}^t \sim DP(\alpha_{0}^t , G_{0}^t) $$


On doit simuler pour l'inititalisation des paramètres :
$$ \xi \sim Gamma(10,1) $$ 
Pour chaque temps : 
$$ \forall t \in T $$
$$ \eta^t \sim Gamma(10,1) $$
$$ \alpha_{0}^t \sim Gamma(10,1) $$


# Initialize parameters

$$ \xi \sim Gamma(10,1) $$ 

In [88]:
a_xi=10
b_xi=1
xi=np.random.gamma(a_xi,b_xi)

Pour chaque temps : 
$$ \forall t \in T $$
$$ \gamma^t \sim Gamma(10,1) $$
$$ \alpha_{0}^t \sim Gamma(10,1) $$


In [87]:
a_gamma=10
b_gamma=1
a_alpha=10
b_alpha=1

gamma=[np.random.gamma(a_gamma,b_gamma) for i in range(T)]
alpha=[np.random.gamma(a_alpha,b_alpha) for i in range(T)]


On crée les time dependencies $v^t=w^t=a$ avec $a \in {0.1,0.3,0.5,0.7,0.9}$ et nous étudierons l'impact de cette variable. 

In [86]:
#ici a=0.5
v=T*[K*[0.5]]
w=T*[0.5]

# Generate from stick breaking for initilization of measures

$$ H \sim Dir ( 1/W) $$ 
$$ G \sim DP(\xi , H) $$

$$ G = \sum_{k=1}^{\infty} \nu_k \delta_{\phi_k} $$
où : 
$$ \nu \sim GEM(\xi) $$ et: $$ \phi_k \sim H $$

In [50]:
def stick_breaking(alpha, k, size_W):
    if(alpha < 0): return("alpha must be positive")
    betas = np.random.beta(1, alpha, k)
    produit_1_beta = np.append(1, np.cumprod(1 - betas[:-1]))
    p = betas * produit_1_beta
    return(p/p.sum())

In [96]:
nu=stick_breaking(xi,K,W)
nu

array([0.15550443, 0.0144072 , 0.1913145 , 0.00791215, 0.10375269,
       0.12049114, 0.00263679, 0.09692101, 0.03604338, 0.01144353,
       0.00846962, 0.01010965, 0.00496922, 0.01472778, 0.00751733,
       0.01104364, 0.01646648, 0.01999952, 0.01635499, 0.0512214 ,
       0.0140396 , 0.01596977, 0.00376132, 0.0099378 , 0.00236992,
       0.00324503, 0.00523835, 0.00136688, 0.00258233, 0.00118158,
       0.00752589, 0.00567227, 0.00214797, 0.00047164, 0.00047127,
       0.00457155, 0.00883168, 0.00175988, 0.00671852, 0.00083029])

$$ G = \sum_{k=1}^{\infty} \nu_k \delta_{\phi_k} $$

Une fois G simulé, on sait que :
$$ G_{0}^t = Dir( \gamma^{t} , G) $$
D'après l'approche stick breaking :
$$ G_{0}^t = \sum_{k=1}^\infty \beta_{k}^t\delta_{\phi_k} $$
où
$$ \beta^t \sim DP(\gamma^t,\hat{\beta}^t)$$$$\hat{\beta}^t=w^t\beta^{t-1}+(1-w^t)\nu$$$$  \nu\sim GEM(\xi) $$ 
Pour simuler $ G_{0}^t$, on défini les deux propriétés suivantes : <br/> <br/>
I. D'après la **propriété de normalisation**  d'un processus de Dirichlet: <br/><br/>
**Si** $$ (X_1,...,X_d)\sim Dirichlet(\alpha_1,...,\alpha_d) $$
**Alors, pour k $\leq$ d**
$$ \dfrac{(X_1,...,X_k)}{\sum_{i\leq k}X_i} \sim Dirichlet(\alpha_1,...,\alpha_k) $$
<br/><br/>
II. Lien entre **la loi de Dirichlet et la loi Beta**:<br/><br/>
**Si** $$(X_1,...,X_d)\sim Dir(\alpha_1,...,\alpha_d)$$ 
**Alors** $$\forall i \in [1,d],
X_i \sim Beta(\alpha_i,\alpha-\alpha_i),\alpha=\sum_{j=1}^d\alpha_j$$ <br/><br/>

Ainsi on a : <br/>
$$ \frac{\beta_k^t}{1-\sum_{i<k}\beta_i^t} \sim Beta(\gamma^t\hat{\beta}_k,\gamma^t(1-\sum_{i\leq k}\hat{\beta}_i))$$<br/>
Stick-Breaking donne :
$$ \tilde{\beta_k^t}/\hat{\beta}_1,...,\hat{\beta}_k  \sim Beta(\gamma^t\hat{\beta}_k,\gamma^t(1-\sum_{i\leq k}\hat{\beta}_i)) $$ i<=k pose problème pour la dernière simulation du K. Du au fait que K est fixé
$$ \beta_k^t=\tilde{\beta_k^t}\prod_{i<k}(1-\tilde{\beta_i^t})$$<br\><br\>



**Propriété des lois Beta/Dirichlet pour d=2** <br\><br\>
Si $$(X_1,X_2)\sim Dirichlet(\alpha,\beta)$$ alors $$X_1 \sim Beta(\alpha,\beta)$$
On se sert PAS ENCORE de cette propriété dans la fonction pour simuler selon une dirichlet au lieu d'une Beta
<br\><br\> 
Dans les fonctions suivantes, on peut avoir besoin de la fonction np.random.dirichlet pour simuler nos nouveaux paramètres.
Cette fonction nécéssite que le vecteur donné en paramètre ait des valeurs >0 ce qui n'est pas forcément le cas. <br\> Pour vérifier cette condition, on crée la fonction suivante : "dirichlet_generate_random"

In [54]:
def dirichlet_generate_random(params_dirich):
    if(type(params_dirich)==list):
        params_dirich=np.array(params_dirich)
    liste_indice_non_zero=np.nonzero(params_dirich)
    param_non_zero=params_dirich[params_dirich>0]
    rand_dir=np.random.dirichlet(param_non_zero)
    random_finale=np.zeros((len(params_dirich)))
    random_finale[liste_indice_non_zero]=rand_dir
    return(random_finale)

def beta_generate_random(params_beta):
    if(type(params_beta)==list):
        params_beta=np.array(params_beta)
    if(len(params_beta)!=2):
        print("ERROR, la taille des paramètres pour la simluation d'une beta est supérieur à 2")
    if(params_beta[0]<=0):
        return(0)
    if(params_beta[1]<=0):
        return(1)
    random_final=np.random.beta(params_beta[0],params_beta[1])
    return(random_final)

In [97]:
# voir (8)
def initialize_G_0_t (gamma,nu,T,K,w):
    G_0_t=[]
    for t in range(T):
        if(t==0):
            beta_t=[]
            beta_tilde_t=[]
            for k in range(K):
                params_dirich=[gamma[t]*nu[k],gamma[t]*(1-np.sum(nu[:k+1]))]
                beta_tilde_k_t=beta_generate_random(params_dirich)
                beta_tilde_t.append(beta_tilde_k_t)
                beta_k_t=beta_tilde_k_t*np.product(1-np.array(beta_tilde_t[:k]))
                beta_t.append(beta_k_t)
            G_0_t.append((beta_t/np.sum(beta_t)).tolist())
        else:        
            beta_t=[]
            beta_tilde_t=[]
            beta_hat=w[t]*np.array(G_0_t[t-1])+(1-w[t])*nu
            for k in range(K):
                params_dirich=[gamma[t]*beta_hat[k],gamma[t]*(1-np.sum(beta_hat[:k+1]))]
                beta_tilde_k_t=beta_generate_random(params_dirich)
                beta_tilde_t.append(beta_tilde_k_t)
                beta_k_t=beta_tilde_k_t*np.product(1-np.array(beta_tilde_t[:k]))
                beta_t.append(beta_k_t)
            G_0_t.append((beta_t/np.sum(beta_t)).tolist())
    return(G_0_t)

In [98]:
beta=initialize_G_0_t (gamma,nu,T,K,w)

Maintenant, on simule $G_j^t$ $$ \forall t \in T, \forall j \in J$$  
$$ G_j^t=\sum_{k=1}^{\infty}\pi_{jk}^t\delta_{\phi_k}$$ $$\pi_j^t\sim DP(\alpha_0^t,\hat{\pi}^t_j)$$

$$\hat{\pi}^t_j=v_j^t\pi_j^{t-1}+(1-v_j^t)\beta^t$$


In [99]:
#voir (9)
def initialize_G_j_t (G_0_t,alpha,J,T,K,v):
    G_j_T=[]
    for t in range(T):
        G_j_t=[]
        if(t==0):
            for j in range(J):
                alpha_j_t=[]
                alpha_tilde_j_t=[]
                for k in range(K):
                    params_dirich=[alpha[t]*G_0_t[t][k],alpha[t]*(1-np.sum(G_0_t[t][:k+1]))]
                    alpha_tilde_k_t=beta_generate_random(params_dirich)
                    alpha_tilde_j_t.append(alpha_tilde_k_t)
                    alpha_k_t=alpha_tilde_k_t*np.product(1-np.array(alpha_tilde_j_t[:k]))
                    alpha_j_t.append(alpha_k_t)     
                G_j_t.append((alpha_j_t/(np.sum(alpha_j_t))).tolist())
            G_j_T.append(G_j_t)
        else:
            for j in range(J):
                alpha_j_t=[]
                alpha_tilde_j_t=[]
                alpha_hat=v[t][j]*np.array(G_j_T[t-1][j])+(1-v[t][j])*np.array(G_0_t[t][k])
                for k in range(K):
                    params_dirich=[alpha[t]*alpha_hat[k],alpha[t]*(1-np.sum(alpha_hat[:k+1]))]
                    alpha_tilde_k_t=beta_generate_random(params_dirich)
                    alpha_tilde_j_t.append(alpha_tilde_k_t)
                    alpha_k_t=alpha_tilde_k_t*np.product(1-np.array(alpha_tilde_j_t[:k]))
                    alpha_j_t.append(alpha_k_t)     
                G_j_t.append((alpha_j_t/(np.sum(alpha_j_t))).tolist())
            G_j_T.append(G_j_t)
    return(G_j_T)

In [100]:
pi=initialize_G_j_t(beta,alpha,J,T,K,v)

Une fois qu'on a initialisé les $$\pi_{jk}^t$$ 
On initialise randomly les Z

La fonction "compute_Z_j_t" permet de calculer les probas normalisées d'un doc au temps t, pour le corpus j.<br/>
La fonction "compute_proba_z_i_j_t_is_k" étend ce calcul à tous les temps et corpus.<br/>
La fonction "log_proba_mult" n'est pas utilisée mais peut s'avérer utile pour éviter les arrondis.<br/>

<br/> Pour l'initialisation, Z est calculé sans information à posteriori

In [59]:
def randomly_assign_Z_initialisation(T,J,K,data):
    Z=[]
    n=[]
    for t in range(T):
        Z_t=[]
        n_t=[]
        for j in range(J):
            Z_t_j=list(np.nonzero(np.random.multinomial(1,[1/K]*K,len(data[t][j])))[1])
            Z_t.append(Z_t_j)
            n_t.append(compute_n_t_j(K,Z_t_j))
        Z.append(Z_t)
        n.append(n_t)
    return(Z,n)
#Z,N=randomly_assign_Z_initialisation(T,J,K,data)

On peut maintenant obtenir $n_{jk}^t$ qui est le nombre de documents du corpus j au temps t qui ont été assignés au topic k (i.e # $z_{ij}^t$ : $z_{ij}^t=k$) <br/> 
La fonction "compute_n_t_j" calcule $n_{jk}^t, \forall k \in K$ et retourne une liste de taille K 

In [60]:
def compute_n_t_j(K,liste_des_Z_temps_t_corpus_j):
    n_t_j=[]
    for k in range(K):
        n_t_j.append(liste_des_Z_temps_t_corpus_j.count(k))
    return(n_t_j)

La fonction "compute_T_jk_t_tplus1_et_T_jk_0_tplus1_multinomiale" calule :   <br\> <br\> $$(T_{jk}^{t \Rightarrow t+1},T_{jk}^{0 \Rightarrow t+1}) \sim Multinomiale (T_{jk}^{t+1},[p,1-p]),(22)$$  <br\> avec $$p=\frac{v_j^{t+1}\pi_{jk}^t}{(1-v_j^{t+1})\beta_k^{t+1} + v_j^{t+1}\pi_{jk}^t} $$

In [61]:
def compute_T_jk_t_tplus1_et_T_jk_0_tplus1_multinomiale(T_jk_Tplus1,v_j_Tplus1,pi_jk_t,beta_k_Tplus1):
    if(((1-v_j_Tplus1)*beta_k_Tplus1+v_j_Tplus1*pi_jk_t)!=0):
        p=(v_j_Tplus1*pi_jk_t)/((1-v_j_Tplus1)*beta_k_Tplus1+v_j_Tplus1*pi_jk_t)
    else:
        p=0
    T_jk_t_tplus1,T_jk_0_tplus1=np.random.multinomial(T_jk_Tplus1, [p,1-p])
    return(T_jk_t_tplus1,T_jk_0_tplus1)

def compute_M_jk_t_tplus1_et_M_jk_0_tplus1_multinomiale(M_k_Tplus1,w_Tplus1,beta_k_t,nu_k):
    if(((1-w_Tplus1)*nu_k+w_Tplus1*beta_k_t)!=0):
        q=(w_Tplus1*beta_k_t)/((1-w_Tplus1)*nu_k+w_Tplus1*beta_k_t)
    else:
        q=0
    M_k_t_tplus1,Mk_0_tplus1=np.random.multinomial(M_k_Tplus1, [q,1-q])
    return(M_k_t_tplus1,Mk_0_tplus1)

$$ N_{jk}^t=n_{jk}^t+T_{jk}^{t \Rightarrow t+1}$$
$n_{jk}^t$ est le nombre de document du corpus j assignés au topic k au temps t <br\>
$T_{jk}^{t \Rightarrow t+1}$ est le nombre de tables qui ont été crées avec les menus du temps t. <br\><br\>
Pour estimer $T_{jk}^t$, on a besoin du plus d'informations sur cette variables, soit le nombre de documents constituant ce topic, et le nombre de tables qui par la suite ont été considérée comme issue de notre résultat et transmises au temps d'après.<br\><br\>
Ex: si 300 documents sont dans le topic k, et que 40tables issues de notre résultats ont été transmises au temps d'après, on peut penser que beaucoup de tables sont nécéssaire pour représenter le topic k. La valeur de $ N_{jk}^t$ étant grande (car = 340), le CRP défini ci dessous génèrera beaucoup de tables.

$ T_{jk}^t/\beta_k^t,\pi_{jk}^{t-1},N_{jk}^t\sim $  CRP  $ (\alpha_0^t v_j^t\pi_{jk}^{t-1} + \alpha_0^t(1- v_j^t)\beta_k^t,N_{jk}^t)$

In [62]:
# Generate table assignments for `num_customers` customers, according to
# a Chinese Restaurant Process with dispersion parameter `alpha`.
def chinese_restaurant_process(num_customers, alpha):
    if (num_customers <= 0 or alpha<0) :
        return(0)
    elif(alpha==0):
        #print("alpha == 0")
        return(0)
    else :
        T_jk_t=0
        for i in range(num_customers):        
            if(np.random.rand()<alpha/(alpha+i)):
                T_jk_t+=1
    return(T_jk_t)
T_jk_t=chinese_restaurant_process(100,15)   

#num_customers=248
#alpha_test=alpha[0]
#T_jk_t=chinese_restaurant_process(num_customers,alpha_test)
#T_jk_t

La fonction suivante retourne une liste (dim K) de listes (dim 3) contenant  : $$T_{jk}^{t\Rightarrow t+1},T_{jk}^{0\Rightarrow t+1},T_{jk}^t$$



In [63]:
def compute_T_tP1_T_0t_Tjkt(temps0,tempsT,T_jk_ttp1,n_jk_t,v_j_t,pi_jk_T_moins1,beta_k_T,alpha_t):
    T_3=[]
    Nu=[]
    if(temps0):
        for k in range(len(n_jk_t)):
            Nu_jk_t=n_jk_t[k]+T_jk_ttp1[k]
            Nu.append(Nu_jk_t)
            param_CRP=(alpha_t*beta_k_T[k])
            T_0_jk=chinese_restaurant_process(Nu_jk_t,param_CRP)
            T_3.append([0,T_0_jk,T_0_jk])
    elif(tempsT):
        for k in range(len(n_jk_t)):
            Nu_jk_t=n_jk_t[k]
            Nu.append(Nu_jk_t)
            param_CRP=(alpha_t*v_j_t[k]*pi_jk_T_moins1[k])+(alpha_t*(1-v_j_t[k])*beta_k_T[k])
            T_0_jk=chinese_restaurant_process(Nu_jk_t,param_CRP)
            T_jk_t_tplus1,T_jk_0_tplus1=compute_T_jk_t_tplus1_et_T_jk_0_tplus1_multinomiale(T_0_jk,v_j_t[k],pi_jk_T_moins1[k],beta_k_T[k])
            T_3.append([T_jk_t_tplus1,T_jk_0_tplus1,T_0_jk])
    else:
        for k in range(len(n_jk_t)):
            Nu_jk_t=n_jk_t[k]+T_jk_ttp1[k]
            Nu.append(Nu_jk_t)
            param_CRP=(alpha_t*v_j_t[k]*pi_jk_T_moins1[k])+(alpha_t*(1-v_j_t[k])*beta_k_T[k])
            T_0_jk=chinese_restaurant_process(Nu_jk_t,param_CRP)
            T_jk_t_tplus1,T_jk_0_tplus1=compute_T_jk_t_tplus1_et_T_jk_0_tplus1_multinomiale(T_0_jk,v_j_t[k],pi_jk_T_moins1[k],beta_k_T[k])
            T_3.append([T_jk_t_tplus1,T_jk_0_tplus1,T_0_jk])
    return(T_3,Nu)

Les fonctions suivantes calculent les Métatables :
<br/> <br/> La dimension de Métatable est une liste : T*K*3

La fonction "compute_T_jk_t_tplus1_et_T_jk_0_tplus1_multinomiale" calule :   <br\> <br\> $$(M_{k}^{t \Rightarrow t+1},M_{k}^{0 \Rightarrow t+1}) \sim Multinomiale (M_{k}^{t+1},[q,1-q]),(25)$$  <br\> avec $$q=\frac{w^{t+1}\beta_{k}^t}{(1-w^{t+1})\nu_k + w^{t+1}\beta{k}^t} $$


In [64]:
def compute_M_tP1_T_0t_Tjkt(t,temps0,tempsT,M_k_ttp1,Tables,w_t,beta_tmoins1_k,gamma_t,nu,K):
    M_3=[]
    Tau=[]
    if(temps0):
        for k in range(K):
            Tau_t_k=np.sum(np.array(Tables)[t,:,k,1])+M_k_ttp1[k]
            Tau.append(Tau_t_k)
            param_CRP=(gamma_t*nu[k])
            M_tk=chinese_restaurant_process(Tau_t_k,param_CRP)
            M_3.append([0,M_tk,M_tk])
    elif(tempsT):
        for k in range(K):
            Tau_t_k=np.sum(np.array(Tables)[t,:,k,1])
            Tau.append(Tau_t_k)
            param_CRP=(gamma_t*w_t*beta_tmoins1_k[k])+(gamma_t*(1-w_t)*nu[k])
            M_tk=chinese_restaurant_process(Tau_t_k,param_CRP)
            M_jk_t_tplus1,M_jk_0_tplus1=compute_M_jk_t_tplus1_et_M_jk_0_tplus1_multinomiale(M_tk,w_t,beta_tmoins1_k[k],nu[k])
            M_3.append([M_jk_t_tplus1,M_jk_0_tplus1,M_tk])
    else:
        for k in range(K):
            Tau_t_k=np.sum(np.array(Tables)[t,:,k,1])+M_k_ttp1[k]
            Tau.append(Tau_t_k)
            param_CRP=(gamma_t*w_t*beta_tmoins1_k[k])+(gamma_t*(1-w_t)*nu[k])
            M_tk=chinese_restaurant_process(Tau_t_k,param_CRP)
            M_jk_t_tplus1,M_jk_0_tplus1=compute_M_jk_t_tplus1_et_M_jk_0_tplus1_multinomiale(M_tk,w_t,beta_tmoins1_k[k],nu[k])
            M_3.append([M_jk_t_tplus1,M_jk_0_tplus1,M_tk])
    return(M_3,Tau)


In [65]:
def compute_Tables_Metatables(T,J,v,w,pi,beta,alpha,gamma,nu,n,K):
    Table=[]
    MetaTable=[]
    Nu=[]
    for t in range(T-1,-1,-1):
        T_t=[]
        Nu_t=[]
        temps_info=t
        if (temps_info==T-1): 
            for j in range(J):
                T_tj,Nu_t_j=compute_T_tP1_T_0t_Tjkt(0,1,0,n[t][j],v[t],pi[t-1][j],beta[t],alpha[t])
                T_t.append(T_tj)
                Nu_t.append(Nu_t_j)
        elif(temps_info==0):
            for j in range(J):
                T_tj,Nu_t_j=compute_T_tP1_T_0t_Tjkt(1,0,np.array(Table)[T-t-2,j,:,0],n[t][j],v[t],0,beta[t],alpha[t])
                T_t.append(T_tj)
                Nu_t.append(Nu_t_j)
        else:
            for j in range(J):
                T_tj,Nu_t_j=compute_T_tP1_T_0t_Tjkt(0,0,np.array(Table)[T-t-2,j,:,0],n[t][j],v[t],pi[t-1][j],beta[t],alpha[t])
                Nu_t.append(Nu_t_j)
                T_t.append(T_tj)   
        Table.append(T_t)
        Nu.append(Nu_t)
    Nu=Nu[::-1]
    Table=Table[::-1]
    Tau=[]
    for t in range(T-1,-1,-1):
        temps_info=t
        if (temps_info==T-1):
            M_t,Tau_t=compute_M_tP1_T_0t_Tjkt(t,0,1,0,Table,w[t],beta[t-1],gamma[t],nu,K)
            MetaTable.append(M_t)
            Tau.append(Tau_t)
        elif(temps_info==0):
            M_t,Tau_t=compute_M_tP1_T_0t_Tjkt(t,1,0,np.array(MetaTable)[T-2-t,:,0],Table,w[t],0,gamma[t],nu,K)
            MetaTable.append(M_t)
            Tau.append(Tau_t)
        else:
            M_t,Tau_t=compute_M_tP1_T_0t_Tjkt(t,0,0,np.array(MetaTable)[T-2-t,:,0],Table,w[t],beta[t-1],gamma[t],nu,K)
            MetaTable.append(M_t)
            Tau.append(Tau_t)
    Tau=Tau[::-1]
    MetaTable=MetaTable[::-1]
    return(Table,MetaTable,Tau,Nu)
            
#Tables,MetaTable,Tau,Nu=compute_Tables_Metatables(T,J,v,w,pi,beta,alpha,gamma,nu,N,len(beta[1])) 



# Sampling $\nu$

Une fois les Tables et Metatables calculés, on va réapproximer les poids. <br/> <br/> $$M_k = \sum_t M_{k}^t$$
<br/> $$M = \sum_k M_{k}$$

$$G/\xi,H,( M_k )_{k=1}^{K} \sim DP(\xi+M,\frac{H+\sum_{k=1}^K M_k \delta_{\phi_k}}{\xi + M})$$
où K est le nombre de plats distincts sur toutes les métatables. On peut représenter G de la façon suivante :
$$ G = \sum_{k=1}^K \nu_k \delta_{\phi_k} + \nu_u G_u$$$$ G_u\sim DP(\xi,H) $$$$ \nu=(\nu_1,...,\nu_K,\nu_u)\sim Dirichlet(M_1,...,M_K,\xi)$$ <br\>On simule donc $\nu$ selon une loi de dirichlet de paramètres $M_1,...,M_k,M_u$ 

**Calcul de $\beta$ ** <br\>
Une fois qu'on a réduit les dimensions de nos objets et conservé seulement les topics intéressants, on sample $\beta^t$ selon 14 <br\> 
$$(\beta_u^t,\beta_1^t,...,\beta_K^t)\sim Dirichlet(\tilde{\gamma^t}.(\tilde{\beta_u^t},\tilde{\beta_1^t},...,\tilde{\beta_K^t}))$$ avec 

$$\tilde{\gamma^t}=\gamma^t+  TAU^t_.$$ et 
$$ \tilde{\beta_k^t} = \frac{1}{\tilde{\gamma^t}}(\gamma^t w^t \beta_k^{t-1} + \gamma^t(1 - w^t)\nu_k+ TAU^t_k)$$

<br\>
et
<br\>

$$ \tilde{\beta_u^t} = \frac{1}{\tilde{\gamma^t}}(\gamma^t w^t \beta_u^{t-1} + \gamma^t(1 - w^t)\nu_u)$$


Tau et Nu sont calculés plus haut, on s'en sert pour calculé les tildes <br\> Les fonctions suivantes calculent respectivement, $\tilde{\gamma}$ , $\tilde{\beta}$ et $\beta$

In [67]:
def compute_gamma_tilde(gamma,Tau):
    gamma_tilde=gamma+np.sum(np.array(Tau),axis=1)
    return(gamma_tilde)
#gamma_tilde=compute_gamma_tilde(gamma,Tau)


def compute_beta_t_tilde(t,gamma_t,gamma_tilde_t,w_t,beta_tmoins1,nu,tau_t):
    beta_t_tilde=[]
    if(t!=0):
        for k in range(len(nu)-1):
            if(gamma_tilde_t>0):
                beta_t_k_tilde=(1/gamma_tilde_t)*(gamma_t*w_t*beta_tmoins1[k]+gamma_t*(1-w_t)*nu[k]+tau_t[k])
                beta_t_tilde.append(beta_t_k_tilde)
            else:
                beta_t_tilde.append(0) 
        if(gamma_tilde_t>0):
            beta_t_u_tilde=(1/gamma_tilde_t)*(gamma_t*w_t*beta_tmoins1[len(nu)-1]+gamma_t*(1-w_t)*nu[len(nu)-1])
            beta_t_tilde.append(beta_t_u_tilde)
        else: 
            beta_t_tilde.append(0) 
    else:
        for k in range(len(nu)-1):
            if(gamma_tilde_t):
                beta_t_k_tilde=(1/gamma_tilde_t)*(gamma_t*nu[k]+tau_t[k])
                beta_t_tilde.append(beta_t_k_tilde)
            else:
                beta_t_tilde.append(0)  
        if(gamma_tilde_t):        
            beta_t_u_tilde=(1/gamma_tilde_t)*(gamma_t*nu[len(nu)-1])
            beta_t_tilde.append(beta_t_u_tilde)
        else:
            beta_t_tilde.append(0)  

    return(beta_t_tilde)

def compute_new_beta(gamma,w,nu,tau):
    beta_new=[]
    gamma_tilde=compute_gamma_tilde(gamma,tau)
    for t in range(len(gamma_tilde)):
        if(t==0):
            beta_t_tilde=compute_beta_t_tilde(t,gamma[t],gamma_tilde[t],w[t],None,nu,tau[t])
        else:
            beta_t_tilde=compute_beta_t_tilde(t,gamma[t],gamma_tilde[t],w[t],beta_new[t-1],nu,tau[t])
        params_dirich=gamma_tilde[t]*np.array(beta_t_tilde)
        beta_t=dirichlet_generate_random(params_dirich)
        beta_new.append(beta_t.tolist())
    return(beta_new)
#new_beta=compute_new_beta(gamma,w,nu2,Tau)

**Calcul de $\pi$ ** <br\>
De même que pour $\beta$, on calcule $\pi$ de ma façon suivante : <br\>

$$(\pi_{ju}^t,\pi_{j1}^t,...,\pi_{jK}^t)\sim Dirichlet(\tilde{\alpha_{0j}^t}.(\tilde{\pi_{ju}^t},\tilde{\pi_{j1}^t},...,\tilde{\pi_{jK}^t}))$$ avec 

$$\tilde{\alpha}_{0j}^t=\alpha_0^t+  N^t_{j.}$$ et 
$$ \tilde{\pi_{jk}^t} = \frac{1}{\tilde{\alpha_0^t}}(\alpha_0^t v^t \pi_{jk}^{t-1} + \alpha_0^t(1 - v^t)\beta_k^t+ N^t_{jk})$$

<br\>
et
<br\>

$$ \tilde{\pi_{ju}^t} = \frac{1}{\tilde{\alpha_0^t}}(\alpha_0^t v^t \pi_{jk}^{t-1} + \alpha_0^t(1 - v^t)\beta_k^t$$


In [68]:
  
def compute_alpha_tilde(alpha,Nu):
    alpha_tilde=[]
    for j in range(len(Nu[0])):
        alpha_tilde.append((alpha+np.sum(np.array(Nu)[:,j,:],axis=1)).tolist())
    alpha_tilde=np.transpose(np.array(alpha_tilde))
    return(alpha_tilde.tolist())
#alpha_tilde=compute_alpha_tilde(alpha,Nu)

def compute_pi_t_j_tilde(t,j,alpha_0_t,alpha_0_t_tilde,v_t,pi_tmoins1_j,beta_t,Nu_t_j):
    pi_t_j_tilde=[]
    if(t!=0):
        for k in range(len(beta_t)-1):
            if(alpha_0_t_tilde>0):
                pi_t_j_tilde_k=(1/alpha_0_t_tilde)*(alpha_0_t*v_t[k]*pi_tmoins1_j[k]+alpha_0_t*(1-v_t[k])*beta_t[k]+Nu_t_j[k])
                pi_t_j_tilde.append(pi_t_j_tilde_k)
            else:
                pi_t_j_tilde.append(0)
    else:
        for k in range(len(beta_t)-1):
            if(alpha_0_t_tilde>0):
                pi_t_j_tilde_k=(1/alpha_0_t_tilde)*(alpha_0_t*beta_t[k]+Nu_t_j[k])
                pi_t_j_tilde.append(pi_t_j_tilde_k)
            else:
                pi_t_j_tilde.append(0)
    if(alpha_0_t_tilde>0):
        pi_t_j_tilde_u=(1/alpha_0_t_tilde)*(alpha_0_t*beta_t[len(beta_t)-1])
    else:pi_t_j_tilde_u=0
    pi_t_j_tilde.append(pi_t_j_tilde_u)
    return(pi_t_j_tilde)

def compute_new_pi(alpha_0,v,beta,gamma,w,Nu):
    pi_new=[]
    alpha_tilde=compute_alpha_tilde(alpha_0,Nu)
    for t in range(len(alpha_0)):
        pi_new_t=[]
        if(t==0): 
            for j in range(len(Nu[0])):
                    pi_new_t_j_tilde=compute_pi_t_j_tilde(t,j,alpha_0[t],alpha_tilde[t][j],v[t],None,beta[t],Nu[t][j])
                    params_dirich=alpha_tilde[t][j]*np.array(pi_new_t_j_tilde)
                    pi_new_t_j=list(dirichlet_generate_random(params_dirich))
                    pi_new_t.append(pi_new_t_j)
        else:            
            for j in range(len(Nu[0])):
                    pi_new_t_j_tilde=compute_pi_t_j_tilde(t,j,alpha_0[t],alpha_tilde[t][j],v[t],pi_new[t-1][j],beta[t],Nu[t][j])
                    params_dirich=alpha_tilde[t][j]*np.array(pi_new_t_j_tilde)
                    pi_new_t_j=list(dirichlet_generate_random(params_dirich))
                    pi_new_t.append(pi_new_t_j)
        pi_new.append(pi_new_t)
    return(pi_new)
                    
#new_pi=compute_new_pi(alpha,v,beta,gamma,w,Nu)

Si on a plusieurs topics présents par corpus, cette fonction en extrait les plus fréquents. <br\> Cette fonction permet de vérifier nos résultats

In [76]:
def get_best_topic_from_pi(N,average_phi):
    bestAll=[]
    for t in range(len(N)):
        print("----- Temps {} -----:".format(t)) 
        for j in range(len(N[t])):
            print("Corpus {}:".format(j))
            best=np.argsort(-np.array(N[t][j]))
            for i in range(3):
                print("Topic #{}={}" .format(i,average_phi[best[i]]))

On **resample** chaque observation en suivant (20), (21) et l'information à posteriori donnée par (4.5). <br\>
En sortie, on a le topic le plus à même d'étre lié avec l'observation ainsi que la moyenne de chaque topic. 
En effet chaque: $\phi_k \sim Dir(param)$ où param est calculé à posteriori. La moyenne de chaque r.v. nous informe sur le topic et nous permet de faire des comparaisons avec les résultats obtenus en Table 2 de l'article.
<br/> <br/>
On peut calculer $$ P(z_{ji}^t=k / x_{ji}^t)\sim P(z_{ji}^t=k/ \pi_j^t).P(x_{ji}^t/ z_{ji}^t=k...)$$
On sait que $$ P(z_{ji}^t=k/ \pi_j^t) = \pi_{jk}^t $$
De plus, $$ P(x_{ji}^t/ z_{ji}^t=k...) = \frac{\Gamma(n+1) \Gamma (\sum_{a\in A,w}^{W} X_{aw} +\alpha_w)
 \prod_{w=1}^{W} \Gamma (\alpha_w + x_{jiw}^t+ \sum_{a\in A} X_{aw}) }{\Gamma (\sum_{a\in A,w}^{W} X_{aw} +\alpha_w + x_{jiw}^t)  \prod_{w=1}^{W} [\Gamma ( x_{jiw}^t +1) \Gamma (\alpha_w + \sum_{a\in A} X_{aw}) ]} $$
Avec $A = ((i,j,t),Z_{ji}^t=k)$
<br/> 

Après normalisation des $P(z_{ji}^t=k / x_{ji}^t)$, on selectionne un nouveau topic pour chaque document. 
<br\> On retourne aussi la moyenne des $\phi_k \sim Dir(\alpha_1 + \sum_{a\in A} X_{a1},...,\alpha_W + \sum_{a\in A} X_{aW}) $ 

In [101]:
def get_new_z_i_j_t_egal_k(last_iteration,t,j,i,x_i_j_t,X,Z,pi_jt,W):
    proba=[]
    log_proba=[]
    average_phi=[]
    
    Z_with_no_Xijt=copy.deepcopy(Z)
    X_with_no_Xijt=copy.deepcopy(X)
    del Z_with_no_Xijt[t][j][i]
    del X_with_no_Xijt[t][j][i]
    
    for k,pi_jtk in enumerate(pi_jt):
        average_phi_k=[]
        flat_Z=[item for y in Z_with_no_Xijt for x in y for item in x]
        flat_X=[item for y in X_with_no_Xijt for x in y for item in x]
        Z_tij_k=[doc for doc,topic in zip(flat_X,flat_Z) if (topic==k)]
        produit_numerateur=1
        produit_denominateur_1=1
        produit_denominateur_2=1
        a=np.sum(Z_tij_k)
        b=np.sum(x_i_j_t)
        for w in range(W):
            if(len(Z_tij_k)==0):
                c=0
            else:
                c=np.sum(Z_tij_k,axis=0)[w] 
            if(last_iteration):
                if(a>0):
                    average_phi_k.append(((1/W)+c)/a)
                else :
                    average_phi_k.append(0)
                    
            produit_numerateur+=gammaln(x_i_j_t[w]+(1/W)+c)
            produit_denominateur_1+=gammaln((1/W)+c)
            produit_denominateur_2+=gammaln(1+x_i_j_t[w])
        log_proba.append(pi_jt[k]*mpmath.exp((+gammaln(len(x_i_j_t)+1)+gammaln(a+1)+produit_numerateur)-(produit_denominateur_2+produit_denominateur_1+gammaln(a+b))))
        average_phi.append(average_phi_k) 
    somme=sum(log_proba)
    for k,pi_jtk in enumerate(pi_jt):
        log_proba[k]=float(log_proba[k]/somme) 
    max_indice=np.random.choice(len(pi_jt),1,p=log_proba)  
    return(max_indice,average_phi)
#newZ=get_new_z_i_j_t_egal_k(1,0,0,0,data[0][0][0],data,Z,pi[0][0],W)

On resample **toutes** les observations et on obtient les nouveaux N, qui sont les compteurs d'assignation aux topics

In [ ]:
def get_new_Z(data,pi,Z,W,K):
    T=np.random.permutation(len(data))
    for t in T:
        J=np.random.permutation(len(data[t]))
        print('Temps{}'.format(t))
        for j in J:
            I=np.random.permutation(len(data[t][j]))
            for i in I:
                if(t==(len(data)-1) and j==(len(data[t])-1) and i==(len(data[t][j])-1)):
                    Z[t][j][i],average_phi=get_new_z_i_j_t_egal_k(1,t,j,i,data[t][j][i],data,Z,pi[t][j],W)
                else:
                    Z[t][j][i],unused_var=get_new_z_i_j_t_egal_k(0,t,j,i,data[t][j][i],data,Z,pi[t][j],W)
    N=[]
    for t in range(len(data)):
        N_t=[]
        for j in range(len(data[t])):
                N_t.append(compute_n_t_j(K,Z[t][j]))
        N.append(N_t)
    return(Z,N,average_phi)


# ALGORITHME

In [ ]:
def algo_evo_hdp(max_iter,corpora_sizes):
    #----Create Data----#
    true_phi=np.zeros((8,2))
    true_phi[0]=[0.1,0.9]
    true_phi[1]=[0.2,0.8]
    true_phi[2]=[0.3,0.7]
    true_phi[3]=[0.4,0.6]
    true_phi[4]=[0.5,0.5]
    true_phi[5]=[0.6,0.4]
    true_phi[6]=[0.7,0.3]
    true_phi[7]=[0.8,0.2]
    T=4
    J=3
    W=2
    K=30
    info_data=local_components_and_corpora_sizes(T,J,corpora_sizes)    
    data=generate_data_from_mixture_of_multinomials(T,J,info_data,true_phi) 
    #----Initialize Hyperparameters----#
    a_xi=10
    b_xi=1
    xi=np.random.gamma(a_xi,b_xi)
    a_gamma=10
    b_gamma=1
    a_alpha=10
    b_alpha=1
    gamma=[]
    gamma=[np.random.gamma(a_gamma,b_gamma) for i in range(T)]
    alpha=[]
    alpha=[np.random.gamma(a_alpha,b_alpha) for i in range(T)]
    v=T*[K*[0.5]]
    w=T*[0.5]
    #----Initialize parameters----#
    params_loi_H=0.5
    nu=stick_breaking(xi,K,W)
    beta=initialize_G_0_t(gamma,nu,T,K,w)
    pi=initialize_G_j_t(beta,alpha,J,T,K,v)
    Z,N=randomly_assign_Z_initialisation(T,J,K,data)
    #----Iterate Cascaded Gibbs Sampler----#
    for i in range(max_iter):
        print("----ITERATION----:\n{}".format(i))
        Tables,MetaTable,Tau,Nu=compute_Tables_Metatables(T,J,v,w,pi,beta,alpha,gamma,nu,N,K)
        #on conserve seulement les topics qui ont été choisis pour décrire au moins un document.
        M_k=np.sum(np.array(MetaTable)[:,:,2],axis=0)
        M=np.sum(M_k)
        liste_indice=np.nonzero(M_k)
        M_k=M_k[M_k>0]
        param_dir=list(M_k)
        #on ajoute un topic pour l'itération suivante
        param_dir.append(xi)
        nu=dirichlet_generate_random(param_dir)
        beta=compute_new_beta(gamma,w,nu,Tau)
        pi=compute_new_pi(alpha,v,beta,gamma,w,Nu)
        K=len(beta[0])
        v=T*[K*[0.5]]
        Z,N,average=get_new_Z(data,pi,Z,W,K)
    #----Print Mean of topic and N----#
        print('---Beta=---:\n{}'.format(beta))
        print('---Pi=---:\n{}'.format(pi))
        print('---Average---=\n{}\n'.format(average)) 
        print('---N---:\n{}'.format(N))
        get_best_topic_from_pi(N,average)
        print('---Tau---\n{}'.format(Tau))
        print('---Nu---\n{}'.format(Nu))
        print('---Tables---\n{}'.format(Tables))
        print('---MetaTable---\n{}'.format(MetaTable))
max_iter=6 
corpora_sizes=[[500,300,400],[510,320,430],[520,320,430],[530,340,450]]

algo_evo_hdp(max_iter,corpora_sizes)   
    

----ITERATION----:
0
Temps0
Temps1
Temps3
[[0.45181137724550896, 0.5482185628742515], [0.506797520661157, 0.49324380165289256], [0.5317824074074075, 0.46826388888888887], [0.5524173553719008, 0.4476239669421488], [0.47751436781609197, 0.522514367816092], [0.5378333333333334, 0.4622142857142857], [0.4994834710743802, 0.5005578512396694], [0.525547619047619, 0.4745], [0.4990151515151515, 0.5010353535353536], [0.5305528846153846, 0.4694951923076923], [0.5047587719298245, 0.49528508771929824], [0.47912162162162164, 0.5209234234234235], [0.5588908450704225, 0.4411443661971831], [0.4657269503546099, 0.5343085106382979], [0.6760943775100402, 0.323925702811245], [0.529480198019802, 0.47056930693069304], [0.2605945659844742, 0.739408962597036], [0.5250657894736842, 0.4749780701754386], [0.5401024590163934, 0.45993852459016393], [0.4919308943089431, 0.5081097560975609], [0.49764388489208633, 0.5023920863309352], [0.5259269662921349, 0.4741292134831461], [0.5212228260869565, 0.47883152173913046],

Temps0
Temps2


# Conclusion 

Les résultats sont donnés en Table 2. 
L'algorithme est très lent car l'optimisation est faîte par Gibbs Sampling. 
Après un nombre assez faible d'itération (environ 5), on retrouve les topics décrivant chacun des lots de données.
Les résultats peuvent s'avérer approximatifs selon les paramètres donnés. 

Points à améliorer : <br\>
- Optimiser l'algorithme et le nombre de boucles
- Tester le modèle avec un corpus de textes 
- Etudier les articles proposés par Nadi, plus récents et combinant réseaux de neuronnes et topic modelling.